## SpaceX Falcon 9 First Stage Landing Prediction
### Lab 1: Collecting the Data

In [ ]:
# Import required libraries
import requests
import pandas as pd
import numpy as np
import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
# Define helper functions
def getBoosterVersion(data):
    for x in data['rocket']:
        if x:
            response = requests.get("https://api.spacexdata.com/v4/rockets/" + str(x)).json()
            BoosterVersion.append(response['name'])

def getLaunchSite(data):
    for x in data['launchpad']:
        if x:
            response = requests.get("https://api.spacexdata.com/v4/launchpads/" + str(x)).json()
            Longitude.append(response['longitude'])
            Latitude.append(response['latitude'])
            LaunchSite.append(response['name'])

def getPayloadData(data):
    for load in data['payloads']:
        if load:
            response = requests.get("https://api.spacexdata.com/v4/payloads/" + load).json()
            PayloadMass.append(response['mass_kg'])
            Orbit.append(response['orbit'])

def getCoreData(data):
    for core in data['cores']:
        if core['core'] is not None:
            response = requests.get("https://api.spacexdata.com/v4/cores/" + core['core']).json()
            Block.append(response['block'])
            ReusedCount.append(response['reuse_count'])
            Serial.append(response['serial'])
        else:
            Block.append(None)
            ReusedCount.append(None)
            Serial.append(None)

        Outcome.append(str(core['landing_success']) + ' ' + str(core['landing_type']))
        Flights.append(core['flight'])
        GridFins.append(core['gridfins'])
        Reused.append(core['reused'])
        Legs.append(core['legs'])
        LandingPad.append(core['landpad'])

In [ ]:
# Request and parse SpaceX launch data
spacex_url = "https://api.spacexdata.com/v4/launches/past"
response = requests.get(spacex_url)
print("Status code:", response.status_code)

data = pd.json_normalize(response.json())
data.head()

In [ ]:
# Clean and prepare data
data = data[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]
data = data[data['cores'].map(len) == 1]
data = data[data['payloads'].map(len) == 1]
data['cores'] = data['cores'].map(lambda x: x[0])
data['payloads'] = data['payloads'].map(lambda x: x[0])
data['date'] = pd.to_datetime(data['date_utc']).dt.date
data = data[data['date'] <= datetime.date(2020, 11, 13)]

In [ ]:
# Initialize global lists
BoosterVersion, PayloadMass, Orbit, LaunchSite, Outcome = [], [], [], [], []
Flights, GridFins, Reused, Legs, LandingPad = [], [], [], [], []
Block, ReusedCount, Serial, Longitude, Latitude = [], [], [], [], []

In [ ]:
# Apply helper functions
getBoosterVersion(data)
getLaunchSite(data)
getPayloadData(data)
getCoreData(data)

In [ ]:
# Combine into one dataframe
launch_dict = {
    'FlightNumber': list(data['flight_number']),
    'Date': list(data['date']),
    'BoosterVersion': BoosterVersion,
    'PayloadMass': PayloadMass,
    'Orbit': Orbit,
    'LaunchSite': LaunchSite,
    'Outcome': Outcome,
    'Flights': Flights,
    'GridFins': GridFins,
    'Reused': Reused,
    'Legs': Legs,
    'LandingPad': LandingPad,
    'Block': Block,
    'ReusedCount': ReusedCount,
    'Serial': Serial,
    'Longitude': Longitude,
    'Latitude': Latitude
}

launch_df = pd.DataFrame(launch_dict)
launch_df.head()

In [ ]:
# Filter for Falcon 9 only
data_falcon9 = launch_df[launch_df['BoosterVersion'] != 'Falcon 1'].copy()
data_falcon9.loc[:, 'FlightNumber'] = range(1, data_falcon9.shape[0] + 1)
data_falcon9.head()

In [ ]:
# Handle missing values
print("Missing values per column:\n", data_falcon9.isnull().sum())
payload_mean = data_falcon9['PayloadMass'].mean()
data_falcon9['PayloadMass'].replace(np.nan, payload_mean, inplace=True)
print("\nMissing values after replacement:\n", data_falcon9.isnull().sum())

In [ ]:
# Export final dataset
data_falcon9.to_csv('dataset_part_1.csv', index=False)
print("Dataset saved successfully as dataset_part_1.csv")

## Authors
**Prepared by:** Asfha Abraha  
**Based on:** IBM Data Science Capstone – SpaceX Falcon 9 First Stage Landing Prediction  
**Instructor:** Dr. Joseph Santarcangelo, PhD, IBM Developer Skills Network  
© 2025 IBM Corporation. All rights reserved.